In [53]:
import os
import pickle
from contextlib import nullcontext
import torch
import tiktoken
from nanoGPT.model import GPT
from dataclasses import dataclass

In [54]:
out_dir = 'mini-gpt-5-2-1' # ignored if init_from is not 'resume'
start = "How to meet a love of your life?"

In [55]:
num_samples = 5 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 100 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = True

In [56]:
@dataclass
class GPTConfig:
    batch_size: int = 32
    block_size: int = 1024
    vocab_size: int = 50304
    n_layer: int = 6
    n_head: int = 8
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = False
    model_type: str = 'reflex'

In [57]:
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

In [58]:
ckpt_path = os.path.join(out_dir, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)
model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

number of parameters: 81.11M


In [59]:
print("No meta.pkl found, assuming GPT-2 encodings...")
enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)

No meta.pkl found, assuming GPT-2 encodings...


In [60]:
# encode the beginning of the prompt
if start.startswith('FILE:'):
    with open(start[5:], 'r', encoding='utf-8') as f:
        start = f.read()
start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

# run generation
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(decode(y[0].tolist()))
            print('---------------')


How to meet a love of your life?”

This is a matter we’ve created a new world. Here’s the whole thing, as we are here with an individual, a bit of an interesting amount of political pressure. The fact that the word “N” is not an actual thing is a good thing, but we can’t wait for it.

The other thing is that they are a few blocks away. They are the same with everyone else.

After many long years we went on to live a little bit more in the same apartment with a more reliable estimate than a bad story. This was like a problem, but for a moment this change was seen in a different way than the one that’s been called a “Soda.”

Here’s one thing that was also the perfect answer, and unfortunately, we’ve had a lot of useful results, but the news has been going well.

"It was a good idea, I think, just a little bit of a joke. I started this with a story about a job that I could barely draw for, say, a bit, I think that was a matter of curiosity. If you work with a team, or a bad person, do you